## TODO
- Make sure that the types in train and test are identical for the non predictive columns
- Extract a small fraction of the data

### References
- Handling null values
  - https://www.kaggle.com/apryor6/santander-product-recommendation/detailed-cleaning-visualization-python

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
data_dir = '/home/lancif/data/santander-product-recommendation'
train_fname = os.path.join(data_dir, 'train_ver2.csv')
test_fname = os.path.join(data_dir, 'test_ver2.csv')
train = pd.read_csv(train_fname)
test = pd.read_csv(test_fname)

print train.shape
print test.shape

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(13647309, 48)
(929615, 24)


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [4]:
print len(train['ncodpers'])
print len(set(train['ncodpers']))

13647309
956645


In [5]:
train = train.set_index(['ncodpers'])
test = test.set_index(['ncodpers'])

Date

In [6]:
for c in ["fecha_dato", "fecha_alta", "ult_fec_cli_1t"]:
    train[c] = pd.to_datetime(train[c],format="%Y-%m-%d")
    test[c] = pd.to_datetime(test[c],format="%Y-%m-%d")

Cleaning 12th column `indrel_1mes`

In [7]:
def to_integer_str(x):
    try:
        return str(int(float(x)))
    except:
        return str(x)
    
train.indrel_1mes = train.indrel_1mes.astype(str).apply(to_integer_str).astype('category')
test.indrel_1mes = test.indrel_1mes.astype(str).apply(to_integer_str).astype('category')

In [8]:
print train.indrel_1mes.value_counts()
print test.indrel_1mes.value_counts()

1      13490681
nan      149781
3          4350
2          1317
P           874
4           306
Name: indrel_1mes, dtype: int64
1      929565
3          27
nan        23
Name: indrel_1mes, dtype: int64


In [14]:
def to_integer_str(x):
    try:
        return str(int(float(x)))
    except:
        return str(x).strip()

def normalize_int(x):
    try:
        return int(str(x))
    except:
        return np.nan

Normalize canal_entrada (007 -> 7)

In [15]:
print '(train data) Before normalize:', len(train.canal_entrada.unique())
train.canal_entrada = train.canal_entrada.apply(to_integer_str)
print '(train data) After normalize:', len(train.canal_entrada.unique())
test.canal_entrada = test.canal_entrada.apply(to_integer_str)

(train data) Before normalize: 163
(train data) After normalize: 163


In [16]:
nb_test_cols = len(test.columns)
train_feature_cols = train.columns[:nb_test_cols]
assert np.all(test.columns == train_feature_cols)

## TODO: normalize age, antiguedad

### Sampling

In [17]:
small_fraction = 0.1

In [18]:
unique_customer = sorted(list(set(train.index.tolist())))
random_customer = np.random.choice(unique_customer, int(len(unique_customer) * small_fraction), replace=False)

In [19]:
train_small_random = train.loc[random_customer]
test_small_random = test.loc[random_customer]

In [20]:
cutoff_customers = unique_customer[:int(len(unique_customer) * small_fraction)]

In [21]:
train_small_order = train.loc[cutoff_customers]
test_small_order = test.loc[cutoff_customers]

### Save result

In [22]:
target_dir = '../../rsc/data/'
train.to_csv(os.path.join(target_dir, 'train_norm.csv'))
test.to_csv(os.path.join(target_dir, 'test_norm.csv'))

train_small_random.to_csv(os.path.join(target_dir, 'train_small_random.csv'))
test_small_random.to_csv(os.path.join(target_dir, 'test_small_random.csv'))

train_small_order.to_csv(os.path.join(target_dir, 'train_small_order.csv'))
test_small_order.to_csv(os.path.join(target_dir, 'test_small_order.csv'))